# Nhóm 11
Nguyễn Phương Thảo 20170116 

Nguyễn Mai Phương 20170106

Phạm Trung Kiên 20170088

Trần Văn Đạo 20170051
# Formatting Models According to Your Use Case 

Để tiền xử lý dữ liệu cho các công cụ phân tích nâng cao khác nhau của Spark, bạn phải xem xét mục tiêu cuối cùng của mình. Sau đây là một số các yêu cầu đối với cấu trúc dữ liệu đầu vào cho từng tác vụ phân tích nâng cao trong MLlib:

1. Trong hầu hết các thuật toán phân loại và hồi quy, bạn muốn đưa dữ liệu của mình vào trong một cột có loại Double để đại diện cho nhãn và một cột loại Vector (có thể dày hoặc thưa) để đại diện cho các features.
2. Trong trường hợp được đề xuất, bạn muốn đưa dữ liệu của mình vào cột người dùng, cột mục (chẳng hạn như phim hoặc sách) và cột xếp hạng.
3. Trong trường hợp học không có giám sát, một cột kiểu Vector (dày hoặc thưa) là cần thiết để đại diện cho các features.
4. Trong trường hợp phân tích đồ thị, bạn sẽ cần một DataFrame của các đỉnh và một DataFrame của các cạnh.

Cách tốt nhất để lấy dữ liệu ở các định dạng này là thông qua transformers. Transformers là tập các hàm nhận một DataFrame làm đối số và trả về một DataFrame mới. Chương này sẽ tập trung vào những transformer nào có liên quan đến các trường hợp sử dụng cụ thể hơn là cố gắng liệt kê mọi transformers có thể.

## NOTE

Spark cung cấp một số transformer như một phần của package org.apache.spark.ml.feature. Package tương ứng trong Python là pyspark.ml.feature. Những transformer mới liên tục xuất hiện trong Spark MLlib nên sẽ không có danh sách chính xác trong tài liệu dưới đây. Thông tin mới nhất có thể được tìm thấy trên Spark website.

Trước khi tiếp tục, chúng ta sẽ đọc một số tập dữ liệu mẫu khác nhau, mỗi tập dữ liệu có các thuộc tính khác nhau mà chúng ta sẽ thao tác trong chương này:

In [ ]:
sales = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../data/retail-data/by-day/*.csv")\
.coalesce(5)\
.where("Description IS NOT NULL")
#fakeIntDF = spark.read.parquet("../data/simple-ml-integers")
simpleDF = spark.read.json("../data/simple-ml")
#scaleDF = spark.read.parquet("../data/simple-ml-scaling")

In [ ]:
scaleDF = spark.read.parquet("../data/simple-ml-scaling")

In [ ]:
fakeIntDF = spark.read.parquet("../data/simple-ml-integers")

Ngoài dữ liệu bán hàng thực tế này, chúng tôi cũng sẽ sử dụng một số bộ dữ liệu tổng hợp đơn giản. FakeIntDF, simpleDF và scaleDF đều có rất ít hàng. Điều này sẽ cung cấp cho bạn khả năng tập trung vào thao tác dữ liệu chính xác mà chúng tôi đang thực hiện thay vì sự mâu thuẫn khác nhau của bất kỳ tập dữ liệu cụ thể nào. Bởi vì chúng tôi sẽ truy cập vào dữ liệu bán hàng một vài lần, chúng tôi sẽ lưu vào bộ nhớ cache để có thể đọc dữ liệu đó một cách hiệu quả từ bộ nhớ thay vì đọc từ đĩa mỗi khi chúng tôi cần. Hãy cũng kiểm tra một số hàng dữ liệu đầu tiên để hiểu rõ hơn về những gì có trong tập dữ liệu

In [ ]:
sales.cache()
sales.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

## Transformers

Chúng ta đã thảo luận về transformers trong chương trước, nhưng chúng ta có thể review nó ở đây. Transformers là tập các hàm thực hiện convert dữ liệu thô theo một cách nào đó. Điều này có thể là để tạo một biến tương tác mới (từ hai biến khác), để chuẩn hóa một cột hoặc chỉ đơn giản là biến nó thành một Double để trở thành input của một mô hình. Transformers chủ yếu được sử dụng trong quá trình tiền xử lý hoặc tạo tính năng.

Transformer của Spark chỉ bao gồm một phương thức biến đổi do nó sẽ không thay đổi theo dữ liệu đầu vào.

Tokenizer là một ví dụ về transformer. Nó tokenize một chuỗi, phân tách các ký tự và không có gì để học từ dữ liệu; nó chỉ đơn giản là áp dụng một hàm. Chúng ta sẽ thảo luận sâu hơn về tokenizer ở phần sau của chương này, nhưng đây là một đoạn mã nhỏ cho thấy cách tokenizer được xây dựng để accept cột đầu vào, cách nó biến đổi dữ liệu và sau đó là kết quả đầu ra từ biến đổi đó

In [ ]:
#// in Scala
#import org.apache.spark.ml.feature.Tokenizer
#val tkn = new Tokenizer().setInputCol("Description")
#tkn.transform(sales.select("Description")).show(false)

## Estimators for Preprocessing

Một công cụ khác để tiền xử lý là các estimators. Estimators là cần thiết khi một phép biến đổi bạn muốn thực hiện phải được khởi tạo bằng dữ liệu hoặc thông tin về cột đầu vào (thường được lấy bằng cách thực hiện pass qua chính cột đầu vào). Ví dụ: nếu bạn muốn chia tỷ lệ các giá trị trong cột để có giá trị trung bình bằng 0 và phương sai đơn vị, bạn sẽ cần thực hiện chuyển trên toàn bộ dữ liệu để tính toán các giá trị bạn sẽ sử dụng để chuẩn hóa dữ liệu. Trên thực tế, estimators có thể là một transformer được cấu hình theo dữ liệu đầu vào cụ thể của bạn. Nói một cách đơn giản nhất, bạn có thể áp dụng một cách mù quáng một phép biến đổi (loại transformer “thông thường”) hoặc thực hiện một phép biến đổi dựa trên dữ liệu của bạn (loại estimator). Hình 25- 2 là một minh họa đơn giản về một ước lượng phù hợp với một tập dữ liệu đầu vào cụ thể, tạo ra một transformer sau đó áp dụng cho tập dữ liệu đầu vào để thêm một cột mới (của dữ liệu đã biến đổi).
![image.png](attachment:image.png)

Một ví dụ về loại estimator này là StandardScaler, chia tỷ lệ cột đầu vào của bạn theo phạm vi giá trị trong cột đó để có giá trị trung bình bằng 0 và phương sai là 1 trong mỗi dimension. Vì lý do đó, trước tiên nó phải thực hiện biến đổi dữ liệu để tạo transformer. Đây là đoạn code mẫu hiển thị toàn bộ quá trình cũng như kết quả đầu ra:

In [ ]:
#// in Scala
#import org.apache.spark.ml.feature.StandardScaler
#val ss = new StandardScaler().setInputCol("features")
#ss.fit(scaleDF).transform(scaleDF).show(false)

## Transformer Properties

Tất cả các transformer đều yêu cầu bạn chỉ định ở mức tối thiểu inputCol và outputCol, đại diện cho tên cột của đầu vào và đầu ra tương ứng. Bạn đặt chúng với setInputCol và setOutputCol. Có một số giá trị mặc định (bạn có thể tìm thấy chúng trong tài liệu), nhưng cách tốt nhất là bạn nên tự chỉ định chúng theo cách thủ công cho rõ ràng. Ngoài các cột đầu vào và đầu ra, tất cả các transformer đều có các thông số khác nhau mà bạn có thể điều chỉnh (bất cứ khi nào chúng tôi đề cập đến một thông số trong chương này, bạn phải đặt lại nó bằng phương thức set ()). Trong Python, chúng tôi cũng có một phương pháp khác để đặt các giá trị này bằng các keyword argument cho phương thức khởi tạo của đối tượng. Chúng tôi loại trừ chúng khỏi các ví dụ trong chương tiếp theo để nhất quán. Estimator yêu cầu bạn điều chỉnh các transformer với tập dữ liệu cụ thể của bạn và sau đó biến đổi trên đối tượng kết quả

## NOTE

Spark MLlib lưu trữ siêu dữ liệu metadata về các cột mà nó sử dụng trong mỗi DataFrame dưới dạng một thuộc tính trên chính cột đó. Điều này cho phép nó lưu trữ (và chú thích) một cách chính xác rằng một cột Double có thể thực sự đại diện cho một loạt các biến phân loại thay vì các giá trị liên tục. Tuy nhiên, siêu dữ liệu sẽ không hiển thị khi bạn in biểu đồ hoặc DataFrame.

# High-Level Transformers

High-level transformers, chẳng hạn như RFormula mà chúng ta đã thấy trong chương trước, cho phép bạn chỉ định ngắn gọn một số phép biến đổi trong một biến đổi. Chúng hoạt động ở “mức độ cao” và cho phép bạn tránh phải thực hiện từng thao tác hoặc biến đổi dữ liệu. Nói chung, bạn nên cố gắng sử dụng các transformer mức cao nhất có thể, để giảm thiểu rủi ro lỗi và giúp bạn tập trung vào vấn đề thay vì các chi tiết nhỏ hơn trong quá trình thực hiện. Mặc dù điều này không phải lúc nào cũng khả thi, nhưng đó là một mục tiêu tốt.

## RFormula

RFormula là transformer dễ sử dụng nhất khi bạn có dữ liệu đã được định dạng "thông thường". Spark mượn transformer này từ ngôn ngữ R để làm cho việc chỉ định khai báo một tập hợp các biến đổi cho dữ liệu của bạn trở nên đơn giản. Với transformer này, các giá trị có thể là số hoặc categorical và bạn không cần phải trích xuất giá trị từ chuỗi hoặc thao tác chúng theo bất kỳ cách nào. RFormula sẽ tự động xử lý các đầu vào categorical (được chỉ định dưới dạng chuỗi) bằng cách thực hiện một mã hóa one-hot encoding. Tóm lại, one-hot encoding biến đổi một tập hợp các giá trị thành một tập hợp các cột nhị phân xác định xem điểm dữ liệu có từng giá trị cụ thể hay không (chúng ta sẽ thảo luận sâu hơn về one-hot encoding ở phần sau của chương). Với RFormula, các cột số sẽ được chuyển thành Double nhưng sẽ không được mã hóa one-hot. Nếu cột nhãn thuộc loại String, đầu tiên nó sẽ được chuyển thành Double với StringIndexer

### WARNING
Tự động truyền các cột số thành Double mà không cần mã hóa one-hot có một số ý nghĩa quan trọng. Nếu bạn có các biến phân loại categorical có giá trị số, chúng sẽ chỉ được chuyển thành Double, ngầm định một thứ tự. Điều quan trọng là đảm bảo các loại đầu vào tương ứng với biến đổi dự kiến. Nếu bạn có các biến categorical thực sự không có quan hệ thứ tự, chúng sẽ được chuyển thành Chuỗi. Bạn cũng có thể lập chỉ mục các cột theo cách thủ công (xem “Working with Categorical Features”)

RFormula cho phép bạn chỉ định các phép biến đổi của mình trong cú pháp khai báo. Nó rất đơn giản để sử dụng một khi bạn hiểu cú pháp. Hiện tại, RFormula hỗ trợ một tập con giới hạn của các toán tử R mà trên thực tế hoạt động khá tốt cho các phép biến đổi đơn giản. Các toán tử cơ bản là
1. ~ riêng biệt target và các terms
2. + Nối các term; “+ 0” có nghĩa là loại bỏ intercept (điều này có nghĩa là intercept y của đường mà chúng ta sắp xếp sẽ là 0)
3. - Xóa bỏ một term; “- 1” có nghĩa là loại bỏ intercept (điều này có nghĩa là intercept y của đường mà chúng ta sắp xếp sẽ là 0)
4. : Interaction (phép nhân cho các giá trị số hoặc các giá trị phân loại được biến đổi thành giá trị nhị phân)
5. . Tất cả các cột ngoại trừ biến mục tiêu / phụ thuộc

RFormula cũng sử dụng các cột nhãn và tính năng mặc định để gắn nhãn, bạn đoán nó, nhãn và tập hợp các tính năng mà nó xuất ra (đối với học có giám sát). Các nodel được đề cập ở phần sau của chương này theo yêu cầu mặc định các tên cột đó, giúp dễ dàng chuyển DataFrame đã chuyển đổi kết quả thành một mô hình để thực hiện training. Nếu điều này vẫn chưa hợp lý, đừng lo lắng— điều đó sẽ trở nên rõ ràng khi chúng ta thực sự bắt đầu sử dụng các model trong các chương sau.

Sử dụng RFormula trong một ví dụ sau. Trong trường hợp này, chúng tôi muốn sử dụng tất cả các biến có sẵn (.) Và sau đó chỉ định tương tác giữa value1 và color, value2 và color làm các feature bổ sung để generate.

In [ ]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")
supervised.fit(simpleDF).transform(simpleDF).show()

+-----+----+------+------------------+--------------------+-----+
|color| lab|value1|            value2|            features|label|
+-----+----+------+------------------+--------------------+-----+
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| bad|     8|14.386294994851129|(10,[2,3,6,9],[8....|  0.0|
| blue| bad|    12|14.386294994851129|(10,[2,3,6,9],[12...|  0.0|
|green|good|    15| 38.97187133755819|(10,[1,2,3,5,8],[...|  1.0|
|green|good|    12|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
|green| bad|    16|14.386294994851129|(10,[1,2,3,5,8],[...|  0.0|
|  red|good|    35|14.386294994851129|(10,[0,2,3,4,7],[...|  1.0|
|  red| bad|     1| 38.97187133755819|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|     2|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red| bad|    16|14.386294994851129|(10,[0,2,3,4,7],[...|  0.0|
|  red|good|    45| 38.97187133755819|(10,[0,2,3,4,7],[...|  1.0|
|green|good|     1|14.386294994851129|(10,[1,2,3,5,8],[...|  1.0|
| blue| ba

## SQL Transformers
SQLTransformer cho phép bạn tận dụng thư viện khổng lồ của Spark về các thao tác liên quan đến SQL giống như cách bạn thực hiện một phép chuyển đổi MLlib. Bất kỳ câu lệnh SELECT nào bạn có thể sử dụng trong SQL đều là một phép biến đổi hợp lệ. Điều duy nhất bạn cần thay đổi là thay vì sử dụng tên bảng, bạn chỉ nên sử dụng từ khóa TH IS. Bạn có thể sử dụng SQLTransformer nếu bạn muốn mã hóa một số thao tác DataFrame như một bước tiền xử lý hoặc thử các biểu thức SQL khác nhau cho các tính năng trong quá trình điều chỉnh siêu tham số metadata. Cũng lưu ý rằng đầu ra của chuyển đổi này sẽ được nối dưới dạng một cột vào DataFrame đầu ra.

Bạn có thể sử dụng SQLTransformer để thể hiện tất cả các thao tác của bạn trên dạng dữ liệu thô sơ nhất để bạn có thể version các biến thể khác nhau của thao tác dưới dạng transformer. Điều này mang lại cho bạn lợi ích từ việc xây dựng và thử nghiệm các pipelines khác nhau, tất cả chỉ bằng cách hoán đổi các transformer. Sau đây là một ví dụ cơ bản về việc sử dụng SQLTransformer

In [ ]:
from pyspark.ml.feature import SQLTransformer
basicTransformation = SQLTransformer()\
.setStatement("""
SELECT sum(Quantity), count(*), CustomerID
FROM __THIS__
GROUP BY CustomerID
""")
basicTransformation.transform(sales).show()

+-------------+--------+----------+
|sum(Quantity)|count(1)|CustomerID|
+-------------+--------+----------+
|          119|      62|   14452.0|
|          440|     143|   16916.0|
|          630|      72|   17633.0|
|           34|       6|   14768.0|
|         1542|      30|   13094.0|
|          854|     117|   17884.0|
|           97|      12|   16596.0|
|          290|      98|   13607.0|
|          541|      27|   14285.0|
|          244|      31|   16561.0|
|          491|     152|   13956.0|
|          204|      76|   13533.0|
|          493|      64|   16629.0|
|          159|      38|   17267.0|
|         1140|      30|   13918.0|
|           55|      28|   18114.0|
|           88|       7|   14473.0|
|          150|      16|   14024.0|
|          206|      23|   12493.0|
|          138|      18|   15776.0|
+-------------+--------+----------+
only showing top 20 rows



## VectorAssembler
VectorAssembler là một công cụ bạn sẽ sử dụng trong hầu hết mọi pipeline bạn generate. Nó giúp kết hợp tất cả các feature của bạn thành một vector lớn mà bạn có thể chuyển vào một estimator. Nó thường được sử dụng trong bước cuối cùng của quy trình học máy và nhận làm đầu vào một số cột Boolean, Double hoặc Vector. Điều này đặc biệt hữu ích nếu bạn định thực hiện một số thao tác sử dụng nhiều loại transformer khác nhau và cần tập hợp tất cả các kết quả đó lại với nhau. Đầu ra từ đoạn mã sau sẽ làm rõ cách hoạt động của điều này:

In [ ]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler().setInputCols(["int1", "int2", "int3"])
va.transform(fakeIntDF).show()

+----+----+----+------------------------------------+
|int1|int2|int3|VectorAssembler_6bae49278a4e__output|
+----+----+----+------------------------------------+
|   1|   2|   3|                       [1.0,2.0,3.0]|
|   7|   8|   9|                       [7.0,8.0,9.0]|
|   4|   5|   6|                       [4.0,5.0,6.0]|
+----+----+----+------------------------------------+



# Working with Continuous Features
Các continuos feature chỉ là các giá trị trên trục số, từ dương vô cùng đến âm vô cùng. Có hai transformer chung cho các tính năng liên tục. Trước tiên, bạn có thể chuyển đổi các continuos feature thành các feature phân loại thông qua một quy trình được gọi là bucketing hoặc bạn có thể mở rộng quy mô và chuẩn hóa các features của mình theo một số yêu cầu khác nhau. Các transformer này sẽ chỉ hoạt động trên loại Double, vì vậy hãy đảm bảo rằng bạn đã chuyển mọi giá trị số khác thành Double

In [ ]:
contDF = spark.range(20).selectExpr("cast(id as double)")

## Bucketing
Cách tiếp cận đơn giản nhất để bucketing hoặc binning là sử dụng Bucketizer. Điều này sẽ chia một continuos feature nhất định thành các nhóm mà bạn chỉ định. Bạn chỉ định cách tạo nhóm thông qua một mảng hoặc list các giá trị Double. Điều này hữu ích vì bạn có thể muốn đơn giản hóa các tính năng trong tập dữ liệu của mình hoặc đơn giản hóa các biểu diễn của chúng để diễn giải sau này. Ví dụ: hãy tưởng tượng bạn có một cột đại diện cho cân nặng của một người và bạn muốn dự đoán một số giá trị dựa trên thông tin này. Trong một số trường hợp, có thể đơn giản hơn nếu tạo ba nhóm “overweight”, “average” và “underweight”.

Để chỉ định bucket, có thể set các border của nó. Ví dụ: phân chia thành 5.0, 10.0, 250.0 trên contDF của chúng tôi sẽ thực sự không thành công vì chúng tôi không cover được tất cả các phạm vi đầu vào có thể có. Khi chỉ định các bucket point của bạn, các giá trị bạn chuyển vào các phần phân tách phải đáp ứng ba yêu cầu:

1. Giá trị tối thiểu trong mảng phân tách của bạn phải nhỏ hơn giá trị tối thiểu trong DataFrame của bạn.
2. Giá trị tối đa trong mảng phân tách của bạn phải lớn hơn giá trị lớn nhất trong DataFrame của bạn.
3. Bạn cần chỉ định tối thiểu ba giá trị trong mảng phân tách, mảng này tạo ra hai buckets.

Để cover tất cả các phạm vi có thể có, scala.Double.NegativeInfinity có thể là một option phân tách khác, với scala.Double.PositiveInfinity để bao gồm tất cả các phạm vi có thể có bên ngoài các phần bên trong. Trong Python, chúng tôi chỉ định điều này theo cách sau: float("inf"), float("-inf")

Để xử lý các giá trị null hoặc NaN, chúng ta phải chỉ định tham số handleInvalid là một giá trị mặc định. Chúng ta có thể giữ các giá trị đó (giữ lại), error hoặc null, hoặc bỏ qua các hàng chứa giá trị đó. Đây là một ví dụ về việc sử dụng bucketing:

In [ ]:
from pyspark.ml.feature import Bucketizer
bucketBorders = [-1.0, 5.0, 10.0, 250.0, 600.0]
bucketer = Bucketizer().setSplits(bucketBorders).setInputCol("id")
bucketer.transform(contDF).show()

+----+-------------------------------+
|  id|Bucketizer_54204d279542__output|
+----+-------------------------------+
| 0.0|                            0.0|
| 1.0|                            0.0|
| 2.0|                            0.0|
| 3.0|                            0.0|
| 4.0|                            0.0|
| 5.0|                            1.0|
| 6.0|                            1.0|
| 7.0|                            1.0|
| 8.0|                            1.0|
| 9.0|                            1.0|
|10.0|                            2.0|
|11.0|                            2.0|
|12.0|                            2.0|
|13.0|                            2.0|
|14.0|                            2.0|
|15.0|                            2.0|
|16.0|                            2.0|
|17.0|                            2.0|
|18.0|                            2.0|
|19.0|                            2.0|
+----+-------------------------------+



Ngoài việc phân tách dựa trên các giá trị được mã hóa cứng, một option khác là phân tách dựa trên các phân vị trong dữ liệu của chúng tôi. Điều này được thực hiện với QuantileDiscretizer, sẽ đưa các giá trị vào các bucket do người dùng chỉ định với các phần tách được xác định bởi các giá trị lượng tử gần đúng. Ví dụ: lượng tử thứ 90 là điểm trong dữ liệu của bạn tại đó 90% dữ liệu nằm dưới giá trị đó. Bạn có thể kiểm soát mức độ phân chia của các nhóm bằng cách đặt sai số tương đối cho phép tính lượng tử gần đúng bằng cách sử dụng setRelativeError. Spark thực hiện điều này bằng cách cho phép bạn chỉ định số lượng buckets bạn muốn từ dữ liệu và nó sẽ chia nhỏ dữ liệu của bạn cho phù hợp. Sau đây là một ví dụ:

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer
bucketer = QuantileDiscretizer().setNumBuckets(5).setInputCol("id")
fittedBucketer = bucketer.fit(contDF)
fittedBucketer.transform(contDF).show()

+----+----------------------------------------+
|  id|QuantileDiscretizer_0955c987ed61__output|
+----+----------------------------------------+
| 0.0|                                     0.0|
| 1.0|                                     0.0|
| 2.0|                                     0.0|
| 3.0|                                     1.0|
| 4.0|                                     1.0|
| 5.0|                                     1.0|
| 6.0|                                     1.0|
| 7.0|                                     2.0|
| 8.0|                                     2.0|
| 9.0|                                     2.0|
|10.0|                                     2.0|
|11.0|                                     3.0|
|12.0|                                     3.0|
|13.0|                                     3.0|
|14.0|                                     3.0|
|15.0|                                     4.0|
|16.0|                                     4.0|
|17.0|                                  

## Advanced bucketing techniques

Các kỹ thuật được mô tả ở đây là những cách phổ biến nhất để bucketing dữ liệu , nhưng có một số cách khác tồn tại trong Spark. Tất cả các quy trình này đều giống nhau từ góc độ luồng dữ liệu: bắt đầu với dữ liệu liên tục và đặt chúng trong các nhóm để chúng trở nên rời rạc. Sự khác biệt phát sinh tùy thuộc vào thuật toán được sử dụng để tính toán các nhóm này. Các ví dụ đơn giản mà chúng ta vừa xem qua rất dễ hiểu và làm việc, nhưng các kỹ thuật nâng cao hơn như locality sensitivity hashing (LSH) cũng có sẵn trong MLlib

# Scaling and Normalization

Chúng tôi đã thấy cách chúng tôi có thể sử dụng bucketing để tạo nhóm từ các biến liên tục. Một nhiệm vụ phổ biến khác là mở rộng quy mô và chuẩn hóa dữ liệu liên tục. Mặc dù không phải lúc nào cũng cần thiết, nhưng làm như vậy thường là cách tốt nhất. Bạn có thể muốn làm điều này khi dữ liệu của bạn chứa một số cột dựa trên các tỷ lệ khác nhau

## StandardScaler

StandardScaler chuẩn hóa một tập hợp các feature để có giá trị trung bình bằng 0 và độ lệch chuẩn là 1. Cờ withStd sẽ chia tỷ lệ dữ liệu thành độ lệch chuẩn đơn vị trong khi cờ withMean (sai so với mặc định) sẽ căn giữa dữ liệu trước khi chia tỷ lệ.

### WARNING
Căn giữa có thể rất tốn kém trên các vector thưa vì nó thường biến chúng thành các vectơ dày, vì vậy hãy cẩn thận trước khi căn giữa dữ liệu của bạn


In [ ]:
from pyspark.ml.feature import StandardScaler
sScaler = StandardScaler().setInputCol("features")
sScaler.fit(scaleDF).transform(scaleDF).show()

+---+--------------+-----------------------------------+
| id|      features|StandardScaler_fb93b8ee2825__output|
+---+--------------+-----------------------------------+
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  0|[1.0,0.1,-1.0]|               [1.19522860933439...|
|  1| [2.0,1.1,1.0]|               [2.39045721866878...|
|  1|[3.0,10.1,3.0]|               [3.58568582800318...|
+---+--------------+-----------------------------------+



## MinMaxScaler
MinMaxScaler sẽ chia tỷ lệ các giá trị trong một vector thành các giá trị tỷ lệ trên thang từ giá trị tối thiểu nhất định đến giá trị tối đa. Nếu bạn chỉ định giá trị nhỏ nhất là 0 và giá trị lớn nhất là 1, thì tất cả các giá trị sẽ nằm trong khoảng từ 0 đến 1

In [ ]:
from pyspark.ml.feature import MinMaxScaler
minMax = MinMaxScaler().setMin(5).setMax(10).setInputCol("features")
fittedminMax = minMax.fit(scaleDF)
fittedminMax.transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MinMaxScaler_b893d8ac3bef__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  0|[1.0,0.1,-1.0]|                    [5.0,5.0,5.0]|
|  1| [2.0,1.1,1.0]|                    [7.5,5.5,7.5]|
|  1|[3.0,10.1,3.0]|                 [10.0,10.0,10.0]|
+---+--------------+---------------------------------+



## MaxAbsScaler
MaxAbsScaler chia tỷ lệ dữ liệu bằng cách chia từng giá trị cho giá trị tối đa của giá trị tuyệt đối trong feature này. Do đó, tất cả các giá trị đều nằm trong khoảng từ −1 đến 1. Transformer này hoàn toàn không thay đổi hoặc căn giữa dữ liệu trong quá trình: 

In [ ]:
from pyspark.ml.feature import MaxAbsScaler
maScaler = MaxAbsScaler().setInputCol("features")
fittedmaScaler = maScaler.fit(scaleDF)
fittedmaScaler.transform(scaleDF).show()

+---+--------------+---------------------------------+
| id|      features|MaxAbsScaler_8505d1d5fdab__output|
+---+--------------+---------------------------------+
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  0|[1.0,0.1,-1.0]|             [0.33333333333333...|
|  1| [2.0,1.1,1.0]|             [0.66666666666666...|
|  1|[3.0,10.1,3.0]|                    [1.0,1.0,1.0]|
+---+--------------+---------------------------------+



## ElementwiseProduct
ElementwiseProduct cho phép chúng tôi chia tỷ lệ từng giá trị trong một vector theo một giá trị tùy ý. Ví dụ: với vectơ bên dưới và hàng “1, 0,1, -1”, đầu ra sẽ là “10, 1,5, -20”. Đương nhiên, kích thước của vector tỷ lệ phải khớp với kích thước của vector bên trong cột có liên quan:

In [ ]:
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.linalg import Vectors
scaleUpVec = Vectors.dense(10.0, 15.0, 20.0)
scalingUp = ElementwiseProduct()\
.setScalingVec(scaleUpVec)\
.setInputCol("features")
scalingUp.transform(scaleDF).show()

+---+--------------+---------------------------------------+
| id|      features|ElementwiseProduct_cecc9720407d__output|
+---+--------------+---------------------------------------+
|  0|[1.0,0.1,-1.0]|                       [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|                       [20.0,16.5,20.0]|
|  0|[1.0,0.1,-1.0]|                       [10.0,1.5,-20.0]|
|  1| [2.0,1.1,1.0]|                       [20.0,16.5,20.0]|
|  1|[3.0,10.1,3.0]|                      [30.0,151.5,60.0]|
+---+--------------+---------------------------------------+



## Normalizer

Bộ chuẩn hóa cho phép chúng ta chia tỷ lệ các vector đa chiều bằng cách sử dụng một trong số các power norm, được đặt thông qua tham số “p”. Ví dụ, chúng ta có thể sử dụng chuẩn Manhattan (hoặc khoảng cách Manhattan) với p = 1, chuẩn Euclid với p = 2, v.v. Khoảng cách Manhattan là thước đo khoảng cách mà bạn chỉ có thể đi từ điểm này đến điểm khác dọc theo các đường thẳng của trục (như các đường phố ở Manhattan).
Đây là một ví dụ về việc sử dụng Normalizer:

In [ ]:
from pyspark.ml.feature import Normalizer
manhattanDistance = Normalizer().setP(1).setInputCol("features")
manhattanDistance.transform(scaleDF).show()

+---+--------------+-------------------------------+
| id|      features|Normalizer_e6de09c9482d__output|
+---+--------------+-------------------------------+
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  0|[1.0,0.1,-1.0]|           [0.47619047619047...|
|  1| [2.0,1.1,1.0]|           [0.48780487804878...|
|  1|[3.0,10.1,3.0]|           [0.18633540372670...|
+---+--------------+-------------------------------+



# Working with Categorical Features

Nhiệm vụ phổ biến nhất đối với các categorical features là lập chỉ mục. Lập chỉ mục chuyển đổi một biến phân loại trong cột thành một biến số mà bạn có thể đưa vào các thuật toán học máy. Mặc dù điều này là đơn giản về mặt khái niệm, nhưng có một số điều quan trọng cần được ghi nhớ để Spark có thể thực hiện điều này một cách ổn định và có thể lặp lại.

Nói chung, chúng tôi khuyên bạn nên lập chỉ mục lại mọi biến phân loại khi tiền xử lý chỉ vì mục đích nhất quán. Điều này có thể hữu ích trong việc duy trì các mô hình của bạn về lâu dài vì các phương pháp mã hóa của bạn có thể thay đổi theo thời gian

## StringIndexer

Cách đơn giản nhất để lập chỉ mục là thông qua StringIndexer, ánh xạ các chuỗi thành các số ID khác nhau. StringIndexer cũng tạo siêu dữ liệu metadata gắn với DataFrame để chỉ định đầu vào nào tương ứng với đầu ra nào. Điều này cho phép chúng tôi sau đó lấy lại đầu vào từ các giá trị chỉ mục tương ứng của chúng

In [ ]:
from pyspark.ml.feature import StringIndexer
lblIndxr = StringIndexer().setInputCol("lab").setOutputCol("labelInd")
idxRes = lblIndxr.fit(simpleDF).transform(simpleDF)
idxRes.show()

+-----+----+------+------------------+--------+
|color| lab|value1|            value2|labelInd|
+-----+----+------+------------------+--------+
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|     1.0|
|  red| bad|     1| 38.97187133755819|     0.0|
|  red| bad|     2|14.386294994851129|     0.0|
|  red| bad|    16|14.386294994851129|     0.0|
|  red|good|    45| 38.97187133755819|     1.0|
|green|good|     1|14.386294994851129|     1.0|
| blue| bad|     8|14.386294994851129|     0.0|
| blue| bad|    12|14.386294994851129|     0.0|
|green|good|    15| 38.97187133755819|     1.0|
|green|good|    12|14.386294994851129|     1.0|
|green| bad|    16|14.386294994851129|     0.0|
|  red|good|    35|14.386294994851129|  


Hãy nhớ rằng StringIndexer là một estimator phải phù hợp với dữ liệu đầu vào. Điều này có nghĩa là nó phải xem xét tất cả các đầu vào để chọn ánh xạ các đầu vào thành ID. Nếu bạn train một StringIndexer trên các đầu vào “a,” “b,” và “c” và sau đó sử dụng nó với đầu vào “d”, nó sẽ xuất hiện một lỗi theo mặc định. Một option khác là bỏ qua toàn bộ hàng nếu giá trị đầu vào không phải là giá trị được xem xét trong quá trình huấn luyện. Cùng với ví dụ trước, giá trị đầu vào là “d” sẽ khiến hàng đó bị bỏ qua hoàn toàn. Chúng tôi có thể đặt option này trước hoặc sau khi training trình chỉ mục hoặc pipeline. Nhiều option hơn có thể được thêm vào tính năng này trong tương lai nhưng kể từ Spark 2.2, bạn chỉ có thể bỏ qua hoặc gặp lỗi trên các đầu vào không hợp lệ

In [ ]:
# valIndexer.setHandleInvalid("skip")
# valIndexer.fit(simpleDF).setHandleInvalid("skip")

## Converting Indexed Values Back to Text

Khi kiểm tra kết quả của quá trình học máy, bạn có thể muốn ánh xạ lại các giá trị ban đầu. Vì các mô hình phân loại MLlib đưa ra dự đoán bằng cách sử dụng các giá trị được lập chỉ mục, nên việc chuyển đổi này rất hữu ích để chuyển đổi các dự đoán của mô hình (chỉ số) trở lại các danh mục ban đầu. Chúng ta có thể làm điều này với IndexToString. Bạn sẽ nhận thấy rằng chúng tôi không phải nhập giá trị của mình vào key String; Spark’s MLlib duy trì siêu dữ liệu này cho bạn. Bạn có thể chọn chỉ định đầu ra.

In [ ]:
from pyspark.ml.feature import IndexToString
labelReverse = IndexToString().setInputCol("labelInd")
labelReverse.transform(idxRes).show()

+-----+----+------+------------------+--------+----------------------------------+
|color| lab|value1|            value2|labelInd|IndexToString_c1e439e978f6__output|
+-----+----+------+------------------+--------+----------------------------------+
|green|good|     1|14.386294994851129|     1.0|                              good|
| blue| bad|     8|14.386294994851129|     0.0|                               bad|
| blue| bad|    12|14.386294994851129|     0.0|                               bad|
|green|good|    15| 38.97187133755819|     1.0|                              good|
|green|good|    12|14.386294994851129|     1.0|                              good|
|green| bad|    16|14.386294994851129|     0.0|                               bad|
|  red|good|    35|14.386294994851129|     1.0|                              good|
|  red| bad|     1| 38.97187133755819|     0.0|                               bad|
|  red| bad|     2|14.386294994851129|     0.0|                               bad|
|  r

## Indexing in Vectors

VectorIndexer là một công cụ hữu ích để làm việc với các biến phân loại đã được tìm thấy bên trong vectơ trong tập dữ liệu của bạn. Công cụ này sẽ tự động tìm các đối tượng phân loại bên trong vectơ đầu vào của bạn và chuyển chúng thành các đối tượng phân loại với các chỉ số danh mục dựa trên số không. Ví dụ: trong DataFrame sau, cột đầu tiên trong Vectơ của chúng tôi là một biến phân loại với hai danh mục khác nhau trong khi phần còn lại của các biến là liên tục. Bằng cách đặt maxCategories thành 2 trong VectorIndexer của chúng tôi, chúng tôi đang hướng dẫn Spark lấy bất kỳ cột nào trong vectơ của chúng tôi có hai hoặc ít giá trị khác biệt hơn và chuyển đổi nó thành một biến phân loại. Điều này có thể hữu ích khi bạn biết có bao nhiêu giá trị duy nhất trong danh mục lớn nhất của mình vì bạn có thể chỉ định điều này và nó sẽ tự động lập chỉ mục các giá trị tương ứng. Ngược lại, Spark thay đổi dữ liệu dựa trên thông số này, vì vậy nếu bạn có các biến liên tục không xuất hiện đặc biệt liên tục (nhiều giá trị lặp lại), chúng có thể được chuyển đổi một cách không chủ ý thành biến phân loại nếu có quá ít giá trị duy nhất

In [ ]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.linalg import Vectors
idxIn = spark.createDataFrame([
(Vectors.dense(1, 2, 3),1),
(Vectors.dense(2, 5, 6),2),
(Vectors.dense(1, 8, 9),3)
]).toDF("features", "label")
indxr = VectorIndexer()\
.setInputCol("features")\
.setOutputCol("idxed")\
.setMaxCategories(2)
indxr.fit(idxIn).transform(idxIn).show()

+-------------+-----+-------------+
|     features|label|        idxed|
+-------------+-----+-------------+
|[1.0,2.0,3.0]|    1|[0.0,2.0,3.0]|
|[2.0,5.0,6.0]|    2|[1.0,5.0,6.0]|
|[1.0,8.0,9.0]|    3|[0.0,8.0,9.0]|
+-------------+-----+-------------+



## One-Hot Encoding

Lập chỉ mục các biến phân loại chỉ là một nửa của câu chuyện. Mã hóa one-hot là một phép biến đổi dữ liệu cực kỳ phổ biến được thực hiện sau khi lập chỉ mục các biến phân loại. Điều này là do lập chỉ mục không phải lúc nào cũng đại diện cho các biến phân loại của chúng ta theo cách chính xác để các downstream models xử lý. Ví dụ: khi chúng tôi lập chỉ mục cho cột "color" của mình, bạn sẽ nhận thấy rằng một số màu có giá trị (hoặc số chỉ mục) cao hơn những màu khác (trong trường hợp của chúng tôi, màu xanh lam blue là 1 và xanh lục green là 2)

Điều này không chính xác vì nó tạo ra biểu diễn toán học mà đầu vào cho thuật toán học máy dường như chỉ định rằng màu xanh lá cây green > màu xanh lam blue, điều này không có ý nghĩa gì trong trường hợp của các categories hiện tại. Để tránh điều này, chúng tôi sử dụng OneHotEncoder, sẽ chuyển đổi từng giá trị riêng biệt thành cờ Boolean (1 hoặc 0) như một thành phần trong vectơ. Khi chúng tôi mã hóa giá trị màu, chúng tôi có thể thấy những giá trị này không còn được sắp xếp theo thứ tự nữa, giúp chúng dễ dàng hơn cho các downstream model (ví dụ: mô hình tuyến tính)

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
lblIndxr = StringIndexer().setInputCol("color").setOutputCol("colorInd")
colorLab = lblIndxr.fit(simpleDF).transform(simpleDF.select("color"))
ohe = OneHotEncoder().setInputCol("colorInd")
ohe = ohe.fit(colorLab)
ohe.transform(colorLab).show()

+-----+--------+----------------------------------+
|color|colorInd|OneHotEncoder_1578c006c426__output|
+-----+--------+----------------------------------+
|green|     1.0|                     (2,[1],[1.0])|
| blue|     2.0|                         (2,[],[])|
| blue|     2.0|                         (2,[],[])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|  red|     0.0|                     (2,[0],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
| blue|     2.0|                         (2,[],[])|
| blue|     2.0|                         (2,[],[])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     1.0|                     (2,[1],[1.0])|
|green|     

# Text Data Transformers

Văn bản luôn là đầu vào phức tạp vì nó thường đòi hỏi nhiều thao tác để ánh xạ tới định dạng mà mô hình học máy sẽ có thể sử dụng hiệu quả. Nói chung, bạn sẽ thấy hai loại văn bản: văn bản dạng tự do và biến phân loại chuỗi. Phần này chủ yếu tập trung vào văn bản dạng tự do vì chúng ta đã thảo luận về các biến phân loại

## Tokenizing Text
Tokenization là quá trình chuyển đổi văn bản dạng tự do thành danh sách các “tokens” hoặc các từ riêng lẻ. Cách dễ nhất để làm điều này là sử dụng lớp Tokenizer. Transformer này sẽ lấy một chuỗi từ, được phân tách bằng khoảng trắng và chuyển chúng thành một mảng từ. Ví dụ: trong tập dữ liệu của mình, chúng tôi có thể muốn chuyển trường Description thành danh sách các token.

In [ ]:
from pyspark.ml.feature import Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn.transform(sales.select("Description"))
tokenized.show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, , water, transfer, tattoos]      |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

Chúng ta cũng có thể tạo một Tokenizer không chỉ dựa trên khoảng trắng mà còn là một biểu thức chính quy với RegexTokenizer. Định dạng của biểu thức chính quy phải tuân theo cú pháp Java Regular Expression (RegEx)

In [ ]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------------------------------+
|Description                        |DescOut                                   |
+-----------------------------------+------------------------------------------+
|RABBIT NIGHT LIGHT                 |[rabbit, night, light]                    |
|DOUGHNUT LIP GLOSS                 |[doughnut, lip, gloss]                    |
|12 MESSAGE CARDS WITH ENVELOPES    |[12, message, cards, with, envelopes]     |
|BLUE HARMONICA IN BOX              |[blue, harmonica, in, box]                |
|GUMBALL COAT RACK                  |[gumball, coat, rack]                     |
|SKULLS  WATER TRANSFER TATTOOS     |[skulls, water, transfer, tattoos]        |
|FELTCRAFT GIRL AMELIE KIT          |[feltcraft, girl, amelie, kit]            |
|CAMOUFLAGE LED TORCH               |[camouflage, led, torch]                  |
|WHITE SKULL HOT WATER BOTTLE       |[white, skull, hot, water, bottle]        |
|ENGLISH ROSE HOT WATER BOTT

Một cách khác để sử dụng RegexTokenizer là sử dụng nó để xuất ra các giá trị khớp với pattern được cung cấp thay vì sử dụng nó như một khoảng trống. Chúng tôi thực hiện điều này bằng cách đặt tham số khoảng trống thành false. Làm điều này với một khoảng trắng dưới dạng một pattern sẽ trả về tất cả các khoảng trắng, điều này không quá hữu ích, nhưng nếu chúng ta đã thực hiện pattern của mình để nắm bắt các từ riêng lẻ, chúng ta có thể trả lại những khoảng trắng đó:

In [ ]:
from pyspark.ml.feature import RegexTokenizer
rt = RegexTokenizer()\
.setInputCol("Description")\
.setOutputCol("DescOut")\
.setPattern(" ")\
.setGaps(False)\
.setToLowercase(True)
rt.transform(sales.select("Description")).show(20, False)

+-----------------------------------+------------------+
|Description                        |DescOut           |
+-----------------------------------+------------------+
|RABBIT NIGHT LIGHT                 |[ ,  ]            |
|DOUGHNUT LIP GLOSS                 |[ ,  ,  ]         |
|12 MESSAGE CARDS WITH ENVELOPES    |[ ,  ,  ,  ]      |
|BLUE HARMONICA IN BOX              |[ ,  ,  ,  ]      |
|GUMBALL COAT RACK                  |[ ,  ]            |
|SKULLS  WATER TRANSFER TATTOOS     |[ ,  ,  ,  ,  ]   |
|FELTCRAFT GIRL AMELIE KIT          |[ ,  ,  ]         |
|CAMOUFLAGE LED TORCH               |[ ,  ]            |
|WHITE SKULL HOT WATER BOTTLE       |[ ,  ,  ,  ,  ]   |
|ENGLISH ROSE HOT WATER BOTTLE      |[ ,  ,  ,  ]      |
|HOT WATER BOTTLE KEEP CALM         |[ ,  ,  ,  ]      |
|SCOTTIE DOG HOT WATER BOTTLE       |[ ,  ,  ,  ]      |
|ROSE CARAVAN DOORSTOP              |[ ,  ]            |
|GINGHAM HEART  DOORSTOP RED        |[ ,  ,  ,  ]      |
|STORAGE TIN VINTAGE LEAF      

## Removing Common Words

Một nhiệm vụ phổ biến sau khi mã hóa là lọc các từ dừng, các từ phổ biến không có liên quan trong nhiều loại phân tích và do đó cần được loại bỏ. Các từ dừng thường xuất hiện trong tiếng Anh bao gồm “the,” “and” và “but.” Spark chứa danh sách các từ dừng mặc định mà bạn có thể thấy bằng cách gọi phương thức sau, có thể phân biệt chữ hoa chữ thường nếu cần (kể từ Spark 2.2, các ngôn ngữ được hỗ trợ cho từ dừng là “danish,” “dutch,” “english,” “finnish,” “french,”
“german,” “hungarian,” “italian,” “norwegian,” “portuguese,” “russian,” “spanish,” “swedish,”
và “turkish”)

In [ ]:
from pyspark.ml.feature import StopWordsRemover
englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover()\
.setStopWords(englishStopWords)\
.setInputCol("DescOut")
stops.transform(tokenized).show()

+--------------------+--------------------+-------------------------------------+
|         Description|             DescOut|StopWordsRemover_8cda260ef3e0__output|
+--------------------+--------------------+-------------------------------------+
|  RABBIT NIGHT LIGHT|[rabbit, night, l...|                 [rabbit, night, l...|
| DOUGHNUT LIP GLOSS |[doughnut, lip, g...|                 [doughnut, lip, g...|
|12 MESSAGE CARDS ...|[12, message, car...|                 [12, message, car...|
|BLUE HARMONICA IN...|[blue, harmonica,...|                 [blue, harmonica,...|
|   GUMBALL COAT RACK|[gumball, coat, r...|                 [gumball, coat, r...|
|SKULLS  WATER TRA...|[skulls, , water,...|                 [skulls, , water,...|
|FELTCRAFT GIRL AM...|[feltcraft, girl,...|                 [feltcraft, girl,...|
|CAMOUFLAGE LED TORCH|[camouflage, led,...|                 [camouflage, led,...|
|WHITE SKULL HOT W...|[white, skull, ho...|                 [white, skull, ho...|
|ENGLISH ROSE HO

Lưu ý cách từ of được xóa trong cột đầu ra. Đó là bởi vì nó là một từ phổ biến đến mức không liên quan đến bất kỳ thao tác downstream nào và chỉ thêm nhiễu vào tập dữ liệu của chúng tôi.

## Creating Word Combinations
Việc tokenize các chuỗi và lọc các từ dừng giúp chúng tôi có một tập hợp các từ sạch để sử dụng làm feature. Người ta thường quan tâm đến sự kết hợp của các từ, thường là bằng cách xem các từ được sắp xếp theo vị trí. Các tổ hợp từ về mặt kỹ thuật được gọi là n-gram — nghĩa là, chuỗi các từ có độ dài n. Một n-gam chiều dài 1 được gọi là đơn vị; những thứ có độ dài 2 được gọi là bigram, và độ dài 3 được gọi là trigram (bất cứ thứ gì ở trên chúng chỉ là 4-gram, 5-gram, v.v.), Thứ tự quan trọng với việc tạo n gam, vì vậy chuyển một câu có ba từ thành biểu diễn bigram sẽ dẫn đến hai bigram. Mục tiêu khi tạo n-gram là để nắm bắt tốt hơn cấu trúc câu và nhiều thông tin hơn những gì có thể thu thập được bằng cách chỉ nhìn vào tất cả các từ riêng lẻ. Hãy tạo một số n-gram để minh họa khái niệm này

Với n-gram, chúng ta có thể xem xét chuỗi các từ thường cùng xuất hiện và sử dụng chúng làm đầu vào cho một thuật toán học máy. Những điều này có thể tạo ra các tính năng tốt hơn so với việc chỉ nhìn vào tất cả các từ riêng lẻ (giả sử được tokenize trên một ký tự khoảng trắng):

In [ ]:
from pyspark.ml.feature import NGram
unigram = NGram().setInputCol("DescOut").setN(1)
bigram = NGram().setInputCol("DescOut").setN(2)
unigram.transform(tokenized.select("DescOut")).show(2, False)
bigram.transform(tokenized.select("DescOut")).show(2, False)

+----------------------+--------------------------+
|DescOut               |NGram_37abaeb63aca__output|
+----------------------+--------------------------+
|[rabbit, night, light]|[rabbit, night, light]    |
|[doughnut, lip, gloss]|[doughnut, lip, gloss]    |
+----------------------+--------------------------+
only showing top 2 rows

+----------------------+---------------------------+
|DescOut               |NGram_254deca9717c__output |
+----------------------+---------------------------+
|[rabbit, night, light]|[rabbit night, night light]|
|[doughnut, lip, gloss]|[doughnut lip, lip gloss]  |
+----------------------+---------------------------+
only showing top 2 rows



## Converting Words into Numerical Representations

Sau khi bạn có các feature của từ, đã đến lúc bắt đầu đếm các số lượng từ và kết hợp từ để sử dụng trong các mô hình của chúng tôi. Cách đơn giản nhất là chỉ cần bao gồm số lượng nhị phân của một từ trong một tài liệu nhất định (trong trường hợp của chúng tôi là một hàng). Về cơ bản, chúng tôi đang đo lường xem mỗi hàng có chứa một từ nhất định hay không. Đây là một cách đơn giản để chuẩn hóa kích thước tài liệu và số lần xuất hiện cũng như nhận các feature số cho phép chúng tôi phân loại tài liệu dựa trên nội dung. Ngoài ra, chúng ta có thể đếm các từ bằng cách sử dụng CountVectorizer hoặc cân lại chúng theo mức độ phổ biến của một từ nhất định trong tất cả các tài liệu bằng cách sử dụng phép biến đổi TF-IDF (thảo luận tiếp theo)

In [ ]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer()\
.setInputCol("DescOut")\
.setOutputCol("countVec")\
.setVocabSize(500)\
.setMinTF(1)\
.setMinDF(2)
fittedCV = cv.fit(tokenized)
fittedCV.transform(tokenized).show(10, False)

+-------------------------------+-------------------------------------+---------------------------------------------+
|Description                    |DescOut                              |countVec                                     |
+-------------------------------+-------------------------------------+---------------------------------------------+
|RABBIT NIGHT LIGHT             |[rabbit, night, light]               |(500,[150,185,212],[1.0,1.0,1.0])            |
|DOUGHNUT LIP GLOSS             |[doughnut, lip, gloss]               |(500,[462,463,492],[1.0,1.0,1.0])            |
|12 MESSAGE CARDS WITH ENVELOPES|[12, message, cards, with, envelopes]|(500,[35,41,166],[1.0,1.0,1.0])              |
|BLUE HARMONICA IN BOX          |[blue, harmonica, in, box]           |(500,[10,16,36,352],[1.0,1.0,1.0,1.0])       |
|GUMBALL COAT RACK              |[gumball, coat, rack]                |(500,[228,280,407],[1.0,1.0,1.0])            |
|SKULLS  WATER TRANSFER TATTOOS |[skulls, , water, trans


Mặc dù kết quả đầu ra trông hơi phức tạp, nhưng nó thực sự chỉ là một vectơ thưa chứa tổng kích thước volcabulary, chỉ mục của từ trong vocalbulary và sau đó là số lượng của từ cụ thể đó:

## Term frequency–inverse document frequency

Một cách khác để tiếp cận vấn đề chuyển đổi văn bản thành biểu diễn số là sử dụng thuật ngữ tần số- nghịch đảo tần số (TF – IDF). Nói một cách đơn giản nhất, TF-IDF đo lường tần suất một từ xuất hiện trong mỗi tài liệu, được tính theo số lượng tài liệu mà từ đó xuất hiện trong.

In [ ]:
tfIdfIn = tokenized\
.where("array_contains(DescOut, 'red')")\
.select("DescOut")\
.limit(10)
tfIdfIn.show(10, False)

+---------------------------------------+
|DescOut                                |
+---------------------------------------+
|[gingham, heart, , doorstop, red]      |
|[red, floral, feltcraft, shoulder, bag]|
|[alarm, clock, bakelike, red]          |
|[pin, cushion, babushka, red]          |
|[red, retrospot, mini, cases]          |
|[red, kitchen, scales]                 |
|[gingham, heart, , doorstop, red]      |
|[large, red, babushka, notebook]       |
|[red, retrospot, oven, glove]          |
|[red, retrospot, plate]                |
+---------------------------------------+



Chúng ta có thể thấy một số từ trùng lặp trong các tài liệu này, nhưng những từ này ít nhất cung cấp một biểu diễn giống-chủ đề thô. Bây giờ hãy nhập nó vào TF – IDF. Để làm điều này, chúng tôi sẽ băm từng từ và chuyển nó thành một biểu diễn số, sau đó cân nhắc từng từ trong ngôn ngữ theo tần suất nghịch đảo của tài liệu. Hashing là một quá trình tương tự như CountVectorizer, nhưng không thể thay đổi được — nghĩa là, từ chỉ mục đầu ra của chúng tôi cho một từ, chúng tôi không thể lấy từ đầu vào của mình (nhiều từ có thể ánh xạ tới cùng một chỉ mục đầu ra)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF
tf = HashingTF()\
.setInputCol("DescOut")\
.setOutputCol("TFOut")\
.setNumFeatures(10000)
idf = IDF()\
.setInputCol("TFOut")\
.setOutputCol("IDFOut")\
.setMinDocFreq(2)

In [ ]:
idf.fit(tf.transform(tfIdfIn)).transform(tf.transform(tfIdfIn)).show(10, False)

+---------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|DescOut                                |TFOut                                                |IDFOut                                                                                                           |
+---------------------------------------+-----------------------------------------------------+-----------------------------------------------------------------------------------------------------------------+
|[gingham, heart, , doorstop, red]      |(10000,[52,804,3372,6594,9808],[1.0,1.0,1.0,1.0,1.0])|(10000,[52,804,3372,6594,9808],[0.0,1.2992829841302609,1.2992829841302609,1.2992829841302609,1.2992829841302609])|
|[red, floral, feltcraft, shoulder, bag]|(10000,[50,52,415,6756,8005],[1.0,1.0,1.0,1.0,1.0])  |(10000,[50,52,415,6756,8005],[0.0,0.0,0.0,0.0,0.0])              

(10000,[2591,4291,4456],[1.0116009116784799,0.0,0.0])

Vectơ này được biểu diễn bằng ba giá trị khác nhau: tổng kích thước volcabulary, hàm băm của mỗi từ xuất hiện trong tài liệu và trọng số của từng thuật ngữ đó. Điều này tương tự với đầu ra CountVectorizer.

## Word2Vec

Word2Vec là một công cụ dựa trên học sâu để tính toán biểu diễn vectơ của một tập hợp các từ. Mục đích là có các từ tương tự gần nhau trong không gian vectơ này, vì vậy chúng ta có thể khái quát hóa về các từ đó. Mô hình này dễ đào tạo và sử dụng, đồng thời đã được chứng minh là hữu ích trong một số ứng dụng xử lý ngôn ngữ tự nhiên, bao gồm nhận dạng thực thể, định dạng, phân tích cú pháp, gắn thẻ và dịch máy

In [ ]:
from pyspark.ml.feature import Word2Vec
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
("Hi I heard about Spark".split(" "), ),
("I wish Java could use case classes".split(" "), ),
("Logistic regression models are neat".split(" "), )
], ["text"])
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text",
outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [Hi, I, heard, about, Spark] => 
Vector: [0.02036476763896644,0.00627172663807869,0.0049649406224489216]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.06342963767903191,0.01215339491942099,-0.05633769024695669]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.017420613393187522,0.008711700141429902,-0.013818052783608438]



# Feature Manipulation

Mặc dù gần như mọi transformer trong ML thao tác không gian feature theo một cách nào đó, các thuật toán và công cụ sau đây là các phương tiện tự động để mở rộng các vectơ feature đầu vào hoặc giảm chúng xuống một số kích thước thấp hơn.

## PCA

Principal Components Analysis (PCA) là một kỹ thuật toán học để tìm ra các khía cạnh quan trọng nhất của dữ liệu của chúng ta (các thành phần chính). Nó thay đổi cách biểu diễn các feature của dữ liệu của chúng tôi bằng cách tạo một tập hợp các fearture mới (“các khía cạnh”). Mỗi feature mới là sự kết hợp của các feature ban đầu. Sức mạnh của PCA là nó có thể tạo ra một tập hợp nhỏ các feature có ý nghĩa hơn để đưa vào mô hình của bạn, với chi phí tiềm năng là khả năng diễn giải

Bạn nên sử dụng PCA nếu bạn có tập dữ liệu đầu vào lớn và muốn giảm tổng số feature bạn có. Điều này thường xuất hiện trong phân tích văn bản trong đó toàn bộ không gian feature là rất lớn và nhiều feature phần lớn không liên quan. Sử dụng PCA, chúng tôi có thể tìm thấy các tổ hợp feature quan trọng nhất và chỉ đưa những feature đó vào mô hình học máy của chúng tôi. PCA nhận một tham số 􁽅, chỉ định số lượng các feature đầu ra cần tạo. Nói chung, kích thước này phải nhỏ hơn nhiều so với kích thước của vectơ đầu vào của bạn.

### NOTE

Chọn đúng 􁽅 không phải là chuyện nhỏ và không có phương án nào mà chúng tôi có thể đưa ra. Xem các chương có liên quan trong ESL và ISL để biết thêm thông tin.

In [ ]:
from pyspark.ml.feature import PCA
pca = PCA().setInputCol("features").setK(2)
pca.fit(scaleDF).transform(scaleDF).show(20, False)

+---+--------------+------------------------------------------+
|id |features      |PCA_7e5eb8219922__output                  |
+---+--------------+------------------------------------------+
|0  |[1.0,0.1,-1.0]|[0.0713719499248417,-0.4526654888147805]  |
|1  |[2.0,1.1,1.0] |[-1.6804946984073723,1.2593401322219198]  |
|0  |[1.0,0.1,-1.0]|[0.0713719499248417,-0.4526654888147805]  |
|1  |[2.0,1.1,1.0] |[-1.6804946984073723,1.2593401322219198]  |
|1  |[3.0,10.1,3.0]|[-10.872398139848944,0.030962697060155975]|
+---+--------------+------------------------------------------+



## Interaction

Trong một số trường hợp, bạn có thể có kiến thức miền về các biến cụ thể trong tập dữ liệu của mình. Ví dụ: bạn có thể biết rằng một sự tương tác nhất định giữa hai biến là một biến quan trọng để đưa vào downstream estimator. feature transformer Interaction cho phép bạn tạo tương tác giữa hai biến theo cách thủ công. Nó chỉ nhân hai đối tượng với nhau — điều mà một mô hình tuyến tính điển hình sẽ không làm được cho mọi cặp đối tượng có thể có trong dữ liệu của bạn. Máy biến áp này hiện chỉ có sẵn trực tiếp trong Scala nhưng có thể được gọi từ bất kỳ ngôn ngữ nào bằng RFormula. Chúng tôi khuyến nghị người dùng chỉ sử dụng RFormula thay vì tạo tương tác theo cách thủ công.
## Polynomial Expansion

Khai triển đa thức được sử dụng để tạo các biến tương tác của tất cả các cột đầu vào. Với khai triển đa thức, chúng tôi chỉ định mức độ chúng tôi muốn thấy các tương tác khác nhau. Ví dụ: đối với đa thức bậc 2, Spark lấy mọi giá trị trong feature vector của chúng tôi, nhân nó với mọi giá trị khác trong feature vector và sau đó lưu trữ kết quả dưới dạng feature. Ví dụ: nếu chúng ta có hai feature đầu vào, chúng ta sẽ nhận được bốn feature đầu ra nếu chúng ta sử dụng đa thức bậc hai (2x2). Nếu chúng ta có ba feature đầu vào, chúng ta sẽ nhận được chín feature đầu ra (3x3). Nếu chúng ta sử dụng đa thức bậc ba, chúng ta sẽ nhận được 27 feature đầu ra (3x3x3), v.v. Sự chuyển đổi này hữu ích khi bạn muốn xem các tương tác giữa các feature cụ thể nhưng không nhất thiết phải chắc chắn về những tương tác nào cần xem xét.

### WARNING

Việc khai triển đa thức có thể làm tăng đáng kể không gian feature của bạn, dẫn đến cả chi phí tính toán cao và overfitting. Sử dụng nó một cách thận trọng, đặc biệt là cho các mức độ cao hơn.

In [ ]:
from pyspark.ml.feature import PolynomialExpansion
pe = PolynomialExpansion().setInputCol("features").setDegree(2)
pe.transform(scaleDF).show()

+---+--------------+----------------------------------------+
| id|      features|PolynomialExpansion_380dbe0ba69d__output|
+---+--------------+----------------------------------------+
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  0|[1.0,0.1,-1.0]|                    [1.0,1.0,0.1,0.1,...|
|  1| [2.0,1.1,1.0]|                    [2.0,4.0,1.1,2.2,...|
|  1|[3.0,10.1,3.0]|                    [3.0,9.0,10.1,30....|
+---+--------------+----------------------------------------+



# Feature Selection
Thông thường, bạn sẽ có một loạt các feature có thể có và muốn chọn một tập hợp con nhỏ hơn để sử dụng cho việc training. Ví dụ: nhiều feature có thể tương quan hoặc sử dụng quá nhiều feature có thể dẫn đến overfitting. Quá trình này được gọi là lựa chọn feature. Có một số cách để đánh giá tầm quan trọng của feature sau khi bạn đã train một mô hình nhưng một tùy chọn khác là thực hiện một số lọc thô trước. Spark có một số tùy chọn đơn giản để làm điều đó, chẳng hạn như ChiSqSelector.
## ChiSqSelector

ChiSqSelector tận dụng một test thống kê để xác định các feature không độc lập với nhãn mà chúng tôi đang cố gắng dự đoán và loại bỏ các feature không liên quan. Nó thường được sử dụng với dữ liệu phân loại để giảm số lượng các feature bạn sẽ nhập vào mô hình của mình, cũng như giảm kích thước của dữ liệu văn bản (ở dạng tần số hoặc số lượng). Vì phương pháp này dựa trên kiểm tra Chi-Square, nên có một số cách khác nhau để chúng tôi có thể chọn các tính năng "tốt nhất". Các phương thức là numTopFeatures, được sắp xếp theo giá trị p-value; phân vị, chiếm một tỷ lệ của các feature đầu vào (thay vì chỉ N feature tốt nhất); và fpr, đặt giá trị p-value bị cắt.

Chúng tôi sẽ chứng minh điều này với đầu ra của CountVectorizer được tạo trước đó trong chương này:

In [ ]:
from pyspark.ml.feature import ChiSqSelector, Tokenizer
tkn = Tokenizer().setInputCol("Description").setOutputCol("DescOut")
tokenized = tkn\
.transform(sales.select("Description", "CustomerId"))\
.where("CustomerId IS NOT NULL")
prechi = fittedCV.transform(tokenized)\
.where("CustomerId IS NOT NULL")
chisq = ChiSqSelector()\
.setFeaturesCol("countVec")\
.setLabelCol("CustomerId")\
.setNumTopFeatures(2)

chisq.fit(prechi).transform(prechi)\
.drop("customerId", "Description", "DescOut").show()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 59210)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    se

Py4JError: An error occurred while calling o971.fit

# Advanced Topics
Có một số chủ đề nâng cao xoay quanh transformers và estimators. Ở đây chúng tôi đề cập đến hai transformer phổ biến nhất, bền bỉ nhất cũng như viết các transformer tùy chỉnh.
## Persisting Transformers
Khi bạn đã sử dụng estimator để định cấu hình transformer, có thể hữu ích khi ghi nó vào đĩa và chỉ cần tải nó khi cần thiết (ví dụ: để sử dụng trong một phiên Spark khác). Chúng tôi đã thấy điều này trong chương trước khi chúng tôi duy trì toàn bộ pipeline. Để duy trì một transformer riêng lẻ, chúng tôi sử dụng phương pháp ghi trên transformer được chọn (fitted) (hoặc transformer tiêu chuẩn) và chỉ định vị trí:

In [ ]:
fittedPCA = pca.fit(scaleDF)
fittedPCA.write().overwrite().save("/tmp/fittedPCA")


Sau đó, chúng tôi có thể load lại nó trong:

In [ ]:
from pyspark.ml.feature import PCAModel
loadedPCA = PCAModel.load("/tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

# Conclusion

Chương này đã giới thiệu cơ bản về nhiều phép biến đổi tiền xử lý phổ biến nhất mà Spark có sẵn. Có một số miền cụ thể mà chúng tôi không có đủ chỗ để đề cập (ví dụ: Biến đổi Cosin rời rạc), nhưng bạn có thể tìm thêm thông tin trong tài liệu. Lĩnh vực này của Spark cũng không ngừng phát triển khi cộng đồng phát triển những cái mới.

Một khía cạnh quan trọng khác của bộ công cụ feature engineering này là tính nhất quán. Trong chương trước, chúng ta đã đề cập đến khái niệm đường ống pipeline, một công cụ thiết yếu để đóng gói và train quy trình làm việc ML từ đầu đến cuối. Trong chương tiếp theo, chúng ta sẽ bắt đầu xem xét nhiều loại tác vụ học máy mà bạn có thể có và những thuật toán nào có sẵn cho mỗi tác vụ